In [4]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sematic_dataloader import SegmentationDataset
from Model import SegNet

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\chand\anaconda3\envs\my_env\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
image_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize image if needed
    transforms.ToTensor(),          # Convert image to PyTorch tensor
])

label_transforms = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize label if needed
    transforms.ToTensor(),          # Convert label to tensor
])

In [4]:
image_dir = 'data/png/train'
label_dir = 'data/png/train_labels'

In [5]:
train_data = SegmentationDataset(image_dir=image_dir, label_dir=label_dir, transform= image_transforms, target_transform= label_transforms)

In [6]:
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

In [7]:
val_image_dir = 'data/png/val'
val_label_dir = 'data/png/val_labels'
test_image_dir = 'data/png/test'
test_label_dir = 'data/png/test_labels'

In [8]:
val_data = SegmentationDataset(image_dir=val_image_dir, label_dir=val_label_dir, transform= image_transforms, target_transform= label_transforms)
val_dataloader = DataLoader(val_data, batch_size=16, shuffle=True)
test_data = SegmentationDataset(image_dir=test_image_dir, label_dir=test_label_dir, transform= label_transforms, target_transform= label_transforms)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [9]:
# model = SegNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
device


device(type='cuda')

In [11]:
model = SegNet()#.to(device)

In [12]:
model

SegNet(
  (enc0): EncoderBlock(
    (layers): ModuleList(
      (0): ConvReLU(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): ConvReLU(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (enc1): EncoderBlock(
    (layers): ModuleList(
      (0): ConvReLU(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): ConvReLU(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), 

In [13]:
print(model.parameters())

<generator object Module.parameters at 0x0000018981D9F4C0>


In [ ]:
for param in model.parameters():
    print(param)

In [14]:
# import cv2
# model.eval()  # Set the model to evaluation mode
# 
# # Optionally, load a pretrained model (if available)
# # model.load_state_dict(torch.load('path_to_pretrained_model.pth'))
# 
# # Define the image preprocessing steps
# preprocess = transforms.Compose([
#     transforms.Resize((256, 256)),  # Resize to match the input size expected by the model
#     transforms.ToTensor(),  # Convert image to PyTorch tensor
#     #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization (optional)
# ])
# 
# # Load and preprocess the image
# image_path = 'data/png/train/22678915_15.png'
# input_image = Image.open(image_path).convert('RGB')
# input_tensor = preprocess(input_image).unsqueeze(0)#.to(device)  # Add batch dimension
# 
# # Perform a forward pass with the model
# with torch.no_grad():  # Disable gradient computation for inference
#     output = model(input_tensor)
# 
# # Post-process the output (e.g., apply sigmoid for binary segmentation)
# output = torch.sigmoid(output)
# 
# # Convert output to a numpy array and threshold it for binary segmentation
# output_np = output.squeeze().cpu().numpy()  # Remove batch and channel dimensions
# binary_mask = (output_np > 0.5).astype('uint8')  # Thresholding to get a binary mask
# 
# # Visualize or save the result
# cv2.imshow('Segmentation Mask', binary_mask * 255)  # Multiply by 255 to convert to a visible grayscale image
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [15]:
import torch.nn as nn
import torch.optim as optim
model = SegNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, masks in train_dataloader:
        inputs, masks = inputs.to(device), masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader)}")

Epoch 1, Loss: 0.0
Epoch 2, Loss: 0.0
Epoch 3, Loss: 0.0
Epoch 4, Loss: 0.0
Epoch 5, Loss: 0.0
Epoch 6, Loss: 0.0
Epoch 7, Loss: 0.0
Epoch 8, Loss: 0.0
Epoch 9, Loss: 0.0
Epoch 10, Loss: 0.0


In [16]:
def predict(model, image_path, transform, device):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
    return output.squeeze(0).cpu()

image_path = 'data/png/test/23729035_15.png'
predicted_mask = predict(model, image_path, image_transforms, device)

In [17]:
predicted_mask

tensor([[[-0.2443,  0.1383,  0.0384,  ...,  0.0799, -0.0249,  0.0603],
         [ 0.1279, -0.0193,  0.0870,  ..., -0.1070,  0.1516,  0.2231],
         [-0.0990, -0.0179, -0.2341,  ..., -0.0653,  0.1620,  0.1267],
         ...,
         [-0.1374, -0.3085,  0.1926,  ..., -0.0806,  0.1750,  0.0166],
         [-0.2028, -0.2144,  0.2699,  ..., -0.1293,  0.0052,  0.0779],
         [ 0.0544, -0.0712, -0.2443,  ...,  0.1473,  0.2050,  0.0110]]])

In [18]:
transform = transforms.ToPILImage()
img = transform(predicted_mask)

# display the PIL image
img.show()

In [19]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, masks in dataloader:
            inputs, masks = inputs.to(device), masks.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, masks)
            val_loss += loss.item()
    return val_loss / len(dataloader)

#val_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=False, num_workers=4)

for epoch in range(num_epochs):
    # Training step (as before)
    model.train()
    running_loss = 0.0
    for inputs, masks in train_dataloader:
        inputs, masks = inputs.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_dataloader)
    
    # Validation step
    val_loss = evaluate(model, val_dataloader, criterion, device)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}")
    
    # Save the model if validation loss decreases
    # torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")

Epoch 1, Train Loss: 0.0, Validation Loss: 0.0
Epoch 2, Train Loss: 0.0, Validation Loss: 0.0
Epoch 3, Train Loss: 0.0, Validation Loss: 0.0
Epoch 4, Train Loss: 0.0, Validation Loss: 0.0
Epoch 5, Train Loss: 0.0, Validation Loss: 0.0
Epoch 6, Train Loss: 0.0, Validation Loss: 0.0
Epoch 7, Train Loss: 0.0, Validation Loss: 0.0
Epoch 8, Train Loss: 0.0, Validation Loss: 0.0
Epoch 9, Train Loss: 0.0, Validation Loss: 0.0
Epoch 10, Train Loss: 0.0, Validation Loss: 0.0


In [20]:
model.eval()

SegNet(
  (enc0): EncoderBlock(
    (layers): ModuleList(
      (0): ConvReLU(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): ConvReLU(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (enc1): EncoderBlock(
    (layers): ModuleList(
      (0): ConvReLU(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): ConvReLU(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), 

In [21]:
image = Image.open(image_path).convert("RGB")
image = image_transforms(image).unsqueeze(0).to(device)
with torch.no_grad():
    output = model(image)

In [22]:
output

tensor([[[[-0.1331,  0.0955,  0.0235,  ..., -0.0431,  0.1252,  0.0738],
          [ 0.1198,  0.1264,  0.1045,  ..., -0.1557,  0.1274,  0.1604],
          [-0.1118, -0.0384, -0.1636,  ..., -0.0861,  0.1426,  0.1664],
          ...,
          [-0.2900, -0.2144,  0.0970,  ..., -0.1159,  0.0553, -0.0330],
          [-0.1346, -0.2411,  0.4362,  ..., -0.0933, -0.0568,  0.1016],
          [ 0.0916,  0.0226, -0.1634,  ...,  0.1570,  0.1064,  0.0378]]]],
       device='cuda:0')